In [1]:
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
import os
import time

import glob


# 경로 변경
# os.chdir('/home/owenchoi07/finance_mlops')

In [2]:
glob.glob('./data_crawler/*.csv')

[]

In [17]:
kor_stock_ohlcv = pd.read_csv('data_crawler/kor_stock_ohlcv.csv', dtype = {'ticker':object})
kor_ticker_list = pd.read_csv('data_crawler/kor_ticker_list.csv', dtype = {'ticker':object})
kor_market_cap = pd.read_csv('data_crawler/kor_market_cap.csv', dtype = {'ticker':object})

kor_stock_fundamental = pd.read_csv('data_crawler/kor_stock_fundamental.csv')
snp500 = pd.read_csv('data_crawler/snp500.csv')


/tmp/ipykernel_473873/2732847774.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_market_cap = pd.read_csv('data_crawler/kor_market_cap.csv')
/tmp/ipykernel_473873/2732847774.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_stock_fundamental = pd.read_csv('data_crawler/kor_stock_fundamental.csv')


In [21]:
kor_market_cap_2 = pd.merge(kor_market_cap, kor_ticker_list, 
        on = 'ticker', 
        how = 'left')

In [24]:
kor_market_cap_2.head()

kor_market_cap_2[kor_market_cap_2['ticker'] == '005930']


,date,market_cap,outstanding_shares,ticker,corp_name,market
572165,2018-01-02,329330258194000,129098494,005930,삼성전자,KOSPI
572166,2018-01-03,333203213014000,129098494,005930,삼성전자,KOSPI
572167,2018-01-04,329717553676000,129098494,005930,삼성전자,KOSPI
572168,2018-01-05,336430675364000,129098494,005930,삼성전자,KOSPI
572169,2018-01-08,335785182894000,129098494,005930,삼성전자,KOSPI
...,...,...,...,...,...,...
573530,2023-07-17,437585060915000,5969782550,005930,삼성전자,KOSPI
573531,2023-07-18,429824343600000,5969782550,005930,삼성전자,KOSPI
573532,2023-07-19,428033408835000,5969782550,005930,삼성전자,KOSPI
573533,2023-07-20,423854561050000,5969782550,005930,삼성전자,KOSPI


In [25]:
kor_stock_ohlcv_2 = pd.merge(kor_stock_ohlcv, kor_ticker_list, 
        on = 'ticker', 
        how = 'left')

In [18]:
df1 = kor_stock_ohlcv[kor_stock_ohlcv['date'] == '2023-07-21']
df1 = pd.merge(df1, kor_ticker_list, 
        on = 'ticker', 
        how = 'left')

In [19]:
def rank_per(df, rank_count):
    max_value_1 = df1[df1['market'] == 'KOSPI'].sort_values(by ='price_change_percentage', ascending = False).head(rank_count)
    max_value_2 = df1[df1['market'] == 'KOSDAQ'].sort_values(by ='price_change_percentage', ascending = False).head(rank_count)
    min_value_1 = df1[df1['market'] == 'KOSPI'].sort_values(by ='price_change_percentage', ascending = True).head(rank_count)
    min_value_2 = df1[df1['market'] == 'KOSDAQ'].sort_values(by ='price_change_percentage', ascending = True).head(rank_count)
    
    total = pd.concat([max_value_1, max_value_2, 
               min_value_1, min_value_2
              ])
    total = total.reset_index(drop = True)
    return total

### 주가 등락 하루

In [7]:
df2 = rank_per(df1, 5)


In [8]:
df2[df2['market'] == 'KOSDAQ']

,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker,corp_name,market
5,2023-07-21,3385,7150,3010,3860,100154716,512206106270,93.00,455910,에스케이증권제9호스팩,KOSDAQ
6,2023-07-21,10200,12870,10120,12870,3260154,39664501300,30.00,226330,신테카바이오,KOSDAQ
7,2023-07-21,850,1086,817,1086,18898109,19434172560,29.90,204840,지엘팜텍,KOSDAQ
8,2023-07-21,1622,2145,1530,2145,14951692,29492994350,29.84,084440,유비온,KOSDAQ
9,2023-07-21,20900,27050,20850,26050,2886493,72432105050,22.88,039200,오스코텍,KOSDAQ
15,2023-07-21,13720,13920,9780,9780,1557558,16557138820,-29.94,080580,오킨스전자,KOSDAQ
16,2023-07-21,4610,4760,4405,4450,2074704,9452997095,-17.59,294140,레몬,KOSDAQ
17,2023-07-21,17840,19800,15670,15800,594055,10298655330,-13.00,322180,티라유텍,KOSDAQ
18,2023-07-21,2570,2605,2240,2240,6328884,15417219195,-12.16,121850,코이즈,KOSDAQ
19,2023-07-21,5810,8300,5700,6070,2723695,19192102980,-10.21,224060,더코디,KOSDAQ


In [11]:
df2

,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker,corp_name,market
0,2023-07-21,1069,1389,1060,1389,27158165,35382009557,29.93,007460,에이프로젠,KOSPI
1,2023-07-21,339,432,337,397,58559277,23596024910,16.42,003060,에이프로젠바이오로직스,KOSPI
2,2023-07-21,730,913,730,830,3940168,3335433818,13.70,012170,아센디오,KOSPI
3,2023-07-21,111700,128900,110900,122400,9514146,1152769400900,12.81,001570,금양,KOSPI
4,2023-07-21,9960,11680,9810,10900,3565427,39068255630,11.91,079160,CJ CGV,KOSPI
5,2023-07-21,3385,7150,3010,3860,100154716,512206106270,93.00,455910,에스케이증권제9호스팩,KOSDAQ
6,2023-07-21,10200,12870,10120,12870,3260154,39664501300,30.00,226330,신테카바이오,KOSDAQ
7,2023-07-21,850,1086,817,1086,18898109,19434172560,29.90,204840,지엘팜텍,KOSDAQ
8,2023-07-21,1622,2145,1530,2145,14951692,29492994350,29.84,084440,유비온,KOSDAQ
9,2023-07-21,20900,27050,20850,26050,2886493,72432105050,22.88,039200,오스코텍,KOSDAQ


In [60]:
df2[df2['market'] == 'KOSPI'].pivot(index = 'date', columns = 'corp_name', values = 'price_change_percentage')

corp_name,CJ CGV,HMM,금양,남선알미우,동양우,씨아이테크,아센디오,에이프로젠,에이프로젠바이오로직스,이엔플러스
date,,,,,,,,,,
2023-07-21,11.91,-5.91,12.81,-6.57,-6.29,-8.62,13.7,29.93,16.42,-13.19


In [29]:
kor_stock_ohlcv_005930 = kor_stock_ohlcv_2[kor_stock_ohlcv_2['ticker'] == '005930'].reset_index()

In [30]:
kor_stock_ohlcv_005930

,index,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker,corp_name,market
0,572165,2018-01-02,51380,51400,50780,51020,169485,432677351468,0.12,005930,삼성전자,KOSPI
1,572166,2018-01-03,52540,52560,51420,51620,200270,518345810160,1.18,005930,삼성전자,KOSPI
2,572167,2018-01-04,52120,52180,50640,51080,233909,600531577700,-1.05,005930,삼성전자,KOSPI
3,572168,2018-01-05,51300,52120,51200,52120,189623,490792925116,2.04,005930,삼성전자,KOSPI
4,572169,2018-01-08,52400,52520,51500,52020,167673,435974098536,-0.19,005930,삼성전자,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
1365,573530,2023-07-17,73200,73500,72800,73300,10060049,736819227400,-0.14,005930,삼성전자,KOSPI
1366,573531,2023-07-18,73200,73500,72000,72000,11697900,847564074902,-1.77,005930,삼성전자,KOSPI
1367,573532,2023-07-19,72700,72800,71300,71700,10896412,782147498048,-0.42,005930,삼성전자,KOSPI
1368,573533,2023-07-20,71100,71500,70800,71000,9732730,692487601800,-0.98,005930,삼성전자,KOSPI


### 주가 등락률 일주일

### 주가 등락 한달

### 주가 등락 1년